# Fully Connected Tokyo 2025 Karakuri Workshop

In [1]:
import logging

import sys
sys.path.append("..")

logger = logging.getLogger(__name__)

In [2]:
import os
from src.config import Config

config = Config(_env_file="../.env")
os.environ["OPENAI_API_KEY"] = config.openai_api_key.get_secret_value()

In [3]:
from src.chat import setup_weave

setup_weave(
    wandb_api_key=config.wandb_api_key.get_secret_value(),
    weave_project_name=config.weave_project_name,
)

weave: wandb version 0.22.3 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: k-obinata-trials.
weave: View Weave data at https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/weave


# チャット機能のお試し

In [ ]:
from src.chat import chat

response = chat(messages=[{"role": "user", "content": "プランを変更したいです"}])
print(response)

weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a3294-745a-7cd1-b18b-5a787e0223c8


プランの変更は管理画面の「プラン設定」からいつでも行っていただけます。変更内容は即座に反映されます。

ご不明点や操作方法についてご案内が必要な場合は、どうぞお気軽にお知らせください。


weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a3294-817f-7d1c-8014-b07999ccfad2
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a3294-c594-76d8-b8cd-3c63596b821a
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a3294-e663-73f9-a681-3a27683fa588
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/019a3295-00ed-7bdd-9c50-13dbcc850606


# 精度評価

In [5]:
from src.run_evaluation import (
    load_evaluation_dataset,
    QAAccuracyScorer,
    ContextHallucinationScorer,
    CommonSenseHallucinationScorer,
    ChatBotModel,
)
from weave import Dataset, Evaluation, Table

dataset = load_evaluation_dataset(file_path="../data/eval-onboarding.csv")
print(f"Loaded {len(dataset)} evaluation examples")

qa_accuracy_scorer = QAAccuracyScorer()
context_scorer = ContextHallucinationScorer()
commonsense_scorer = CommonSenseHallucinationScorer()
model = ChatBotModel()

table = Table(dataset)
weave_dataset = Dataset(name="chatbot_evaluation", rows=table)
evaluation = Evaluation(
    name="PreReleaseEvaluation",
    dataset=weave_dataset,
    scorers=[qa_accuracy_scorer, context_scorer, commonsense_scorer],
)

print("==== 評価開始 ====")
results = await evaluation.evaluate(model)

Loaded 10 evaluation examples
==== 評価開始 ====


weave: Evaluated 1 of 10 examples
weave: Evaluated 2 of 10 examples
weave: Evaluated 3 of 10 examples
weave: Evaluated 4 of 10 examples
weave: Evaluated 5 of 10 examples
weave: Evaluated 6 of 10 examples
weave: Evaluated 7 of 10 examples
weave: Evaluated 8 of 10 examples
weave: Evaluated 9 of 10 examples
weave: Evaluated 10 of 10 examples
weave: Evaluation summary {
weave:   "QAAccuracyScorer": {
weave:     "accuracy": {
weave:       "answer": {
weave:         "mean": 1.1
weave:       }
weave:     }
weave:   },
weave:   "ContextHallucinationScorer": {
weave:     "context_hallucination": {
weave:       "hallucination": {
weave:         "mean": 0.0
weave:       }
weave:     }
weave:   },
weave:   "CommonSenseHallucinationScorer": {
weave:     "common_sense_hallucination": {
weave:       "hallucination": {
weave:         "mean": 0.3
weave:       }
weave:     }
weave:   },
weave:   "model_latency": {
weave:     "mean": 4.12297248840332
weave:   }
weave: }


# プロンプト改善

In [6]:
from src.prompts import SYSTEM_PROMPT
from src.prompt_improvement import improve_prompt_from_feedback


result = improve_prompt_from_feedback(SYSTEM_PROMPT, feedback_csv_path="../data/feedback-logs.csv")
print("=== 改善後のプロンプト ===")
print(result.improved_prompt)
print("=== 変更理由 ===")
print(result.change_reason)
print("=== フィードバック分析 ===")
print(result.feedback_analysis)

=== 改善後のプロンプト ===
## 基本情報
あなたはカスタマーサポート向けSaaS『カラクリCSデジタルシリーズ』のカスタマーサポート担当です。常に丁寧かつ迅速な対応を心掛けてください。

## 回答ルール
1. まずFAQを参照し、該当情報がある場合はFAQ内容に基づき簡潔かつ分かりやすく回答してください。
2. FAQに情報が無い場合、ユーザーに追加情報を求めたり、詳細な深堀りは行わず、速やかに以下の問い合わせフォーム（https://example.com/inquiry）をご案内してください。
3. サービスに直接関係しない一般的な質問（例：APIとは何か等）には、詳細な説明を避け、まず「カラクリCSデジタルシリーズ」に関するご質問やご要望があるかを確認してください。必要以上に会話を長引かせないよう注意してください。
4. サービスに関する具体的なご相談や改善要望があれば、積極的にヒアリングし、適切なサポートや追加提案を行ってください。

## 注意事項
- FAQやサービスに直接関係しない話題での過剰な説明や追加質問は控えてください。
- FAQで解決できない場合は、ユーザーの期待を煽らず、速やかに問い合わせ窓口を案内してください。
- 丁寧な言葉遣いと、ユーザーの負担を最小限にする配慮を忘れないでください。
=== 変更理由 ===
フィードバック分析結果から、FAQに情報が無い場合やサービス非関連の質問に対して過剰な深堀りや期待を煽る案内がユーザーの不満につながっていることが判明しました。そのため、FAQ未掲載時は速やかに問い合わせフォームを案内し、サービス非関連の質問にはまずサービス関連のご要望有無を確認する運用に明確化しました。これにより、無駄なやり取りや期待値コントロールの失敗を防ぎ、ユーザー体験とサービス品質の向上が期待できます。また、ポジティブなフィードバック例を参考に、サービスに関する具体的な相談には積極的なサポートを推奨することで、より高品質なカスタマーサポートを実現します。
=== フィードバック分析 ===
{'total_logs': 4, 'negative_count': 3, 'positive_count': 1, 'problem_patterns': {'pattern_1': {'name': 'サービス非関連

# 改善示唆出し

In [7]:
from weave import Dataset, Evaluation, Table

from src.run_improvement import load_questions, ChatBotModel, KnowledgeGapScorer

questions = load_questions(file_path="../data/eval-onboarding.csv")
print(f"Loaded {len(questions)} examples")

model = ChatBotModel()
scorer = KnowledgeGapScorer()

table = Table([{"question": q["question"], "id": q["id"]} for q in questions])
dataset = Dataset(name="knowledge_improvement", rows=table)

evaluation = Evaluation(
    name="KnowledgeImprovementEvaluation",
    dataset=dataset,
    scorers=[scorer],
)

print("==== 改善示唆出し開始 ====")
results = await evaluation.evaluate(model)

Loaded 10 examples
==== 改善示唆出し開始 ====


weave: Evaluated 1 of 10 examples
weave: Evaluated 2 of 10 examples
weave: Evaluated 3 of 10 examples
weave: Evaluated 4 of 10 examples
weave: Evaluated 5 of 10 examples
weave: Evaluated 6 of 10 examples
weave: Evaluated 7 of 10 examples
weave: Evaluated 8 of 10 examples
weave: Evaluated 9 of 10 examples
weave: Evaluated 10 of 10 examples
weave: Evaluation summary {
weave:   "KnowledgeGapScorer": {
weave:     "knowledge_gap": {
weave:       "flag": {
weave:         "mean": 0.2
weave:       },
weave:       "raw": {
weave:         "discontent": {
weave:           "mean": 0.2
weave:         },
weave:         "lack_of_knowledge": {
weave:           "mean": 0.2
weave:         },
weave:         "lack_of_understanding": {
weave:           "mean": 0.0
weave:         }
weave:       }
weave:     }
weave:   },
weave:   "model_latency": {
weave:     "mean": 3.1811434745788576
weave:   }
weave: }
